In [ ]:
# SQL Agent Example
# 
# This notebook demonstrates how to use the SQL Agent for natural language database interactions.
# The SQL Agent can convert natural language queries into SQL operations and return results in plain English.

import os
import sqlite3

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"


In [2]:
import requests

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    # Open a local file in binary write mode
    with open("Chinook.db", "wb") as file:
        # Write the content of the response (the file) to the local file
        file.write(response.content)
    print("File downloaded and saved as Chinook.db")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as Chinook.db


In [3]:
# Quick Example - Set up and ask a simple question
from uuid import uuid4
from sql_agent.database import SQLDatabase
from sql_agent.agent import SQLAgent


async def quick_example():
    # Create database connection and agent
    db = SQLDatabase("sqlite:///Chinook.db")  
    uuid = uuid4()
    agent = SQLAgent(db, str(uuid))
    
    # Ask a simple question
    response = await agent.run("How many customers do we have in total?")
    response = await agent.run("How much is that multipled by 2?")
    return response
    
# Run the example
response = await quick_example()

In [4]:
response.response

'The total number of customers multiplied by 2 is 118.'

In [5]:
response.history

[{'role': 'system',
  'content': 'system_prompt = """\nYou are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct query to run,\nthen look at the results of the query and return the answer. Unless the user\nspecifies a specific number of examples they wish to obtain, always limit your\nquery to at most 5 results.\n\nYou can order the results by a relevant column to return the most interesting\nexamples in the database. Never query for all the columns from a specific table,\nonly ask for the relevant columns given the question.\n\nYou MUST double check your query before executing it. If you get an error while\nexecuting a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the\ndatabase.\n\nTo start you should ALWAYS look at the tables in the database to see what you\ncan query. Do NOT skip this step.\n\nThen you should query the schema of the most relevant tables.\n""'}